In [ ]:
import os
import numpy as np
import cv2

BACKGROUND_PATH = './background/googlemap_sportscheck.png' # perspectiveMap_mensa
DATASET_PATH = './datasets/continuousFrame/sportscheck_300/iou0.6_conf0.5/' # _with_10interval
RESULT_PATH = './output/sportscheck_projection.mp4'

# Input the pixel coordinates of the reference point in the image, for mensa
# SCR = np.array([[396, 525], [688, 389], [1114, 350], [1388, 947]], dtype=np.float64)
# DEST = np.array([[353, 388], [665, 385], [960, 705], [180, 777]], dtype=np.float64)
# For sportscheck
SCR = np.array([[901, 370], [1553, 362], [1776, 843], [259, 500]], dtype=np.float32)
DEST = np.array([[820, 788], [327, 833], [207, 286], [1066, 370]], dtype=np.float32)

def get_img_size(img_path):
    googleMap = cv2.imread(img_path)
    height, width, _ = googleMap.shape
    return width, height

def read_dataset(DATASET_PATH):
    txt_files = [f for f in os.listdir(DATASET_PATH) if f.endswith('.txt')]
    if txt_files:
        file_path = os.path.join(DATASET_PATH, txt_files[0])
        with open(file_path, 'r') as file:
            dataset = [line.strip().split(',') for line in file]
            return dataset
    else:
        return print("No txt files found in the directory.")

class Coordinate:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def centre_pos(self, x1, y1, w, h, cls_id, obj_id):
        cx = int(x1 + w/2)
        cy = int(y1 + h) # (y2-y1)/2+y1 for cy of bbox

        return cx, cy, cls_id, obj_id

    def coordinates_inSatelliteMap(self, src, dest, coordinates):
        MTX = cv2.getPerspectiveTransform(np.float32(src), np.float32(dest))
        coordinates_np = np.array([coordinates], dtype=np.float32).reshape(-1, 1, 2)
        transformed_coordinates = cv2.perspectiveTransform(coordinates_np, MTX)

        return transformed_coordinates
    
def moving_average_trajectory(points, window_size):
    if len(points) < window_size:
        raise ValueError("Window size should be smaller than the length of the trajectory.")

    # Extract x and y coordinates
    x_coords, y_coords = zip(*points)

    # Apply moving average separately to x and y coordinates
    smoothed_x = moving_average(x_coords, window_size)
    smoothed_y = moving_average(y_coords, window_size)

    # Combine the smoothed x and y coordinates into a new trajectory
    smoothed_trajectory = list(zip(smoothed_x, smoothed_y))

    # Add the first and last points of the original trajectory to the smoothed trajectory
    smoothed_trajectory = [(x_coords[0], y_coords[0])] + smoothed_trajectory + [(x_coords[-1], y_coords[-1])]

    return smoothed_trajectory

def moving_average(data, window_size):
    cumsum = np.cumsum(data, dtype=float)
    cumsum[window_size:] = cumsum[window_size:] - cumsum[:-window_size]
    return cumsum[window_size - 1:] / window_size

def visualising_animation(transformed_data_np, color_dict, WIDTH, HEIGHT, fps=24):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(RESULT_PATH, fourcc, fps, (WIDTH, HEIGHT))
    
    frame_numbers = transformed_data_np[:, 0].astype(int)
    max_frame_number = frame_numbers.max()

    trails_dict = {}

    # Create animation
    for frame_number in range(1, max_frame_number + 1):  # 从第1帧开始到最大帧数
        frame_data = transformed_data_np[frame_numbers == frame_number]
        background = cv2.imread(BACKGROUND_PATH)
        
        # 处理当前帧的目标
        current_trails_dict = {}
        for data in frame_data:
            _, x, y, cls_id, obj_id = data
            key = (cls_id, obj_id)  # 创建键
            color = color_dict[cls_id]
            
            # 绘制目标点和文本
            position = (int(float(x)), int(float(y)))
            cv2.circle(background, position, 5, color, -1)
            cv2.putText(background, f'{cls_id} {obj_id}', (position[0] + 10, position[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            
            # 更新或添加轨迹点
            if key not in trails_dict:
                trails_dict[key] = []
            trails_dict[key].append(position)
            
            # 将当前帧的目标添加到current_trails_dict中
            current_trails_dict[key] = trails_dict[key]

        # 绘制所有轨迹
        for key, trail in trails_dict.items():
            for i in range(1, len(trail)):
                cv2.line(background, trail[i - 1], trail[i], color_dict[key[0]], 2)

        # 更新轨迹字典为当前帧的目标
        trails_dict = current_trails_dict
        
        # 显示当前帧
        cv2.imshow('Projective animation', background)
        
        # 写入当前帧到视频文件
        video_writer.write(background)
        
        # 按 'q' 或 'Esc' 键退出
        if cv2.waitKey(1) & 0xFF in [ord('q'), 27]:
            break
    
    # 释放资源并关闭窗口
    video_writer.release()
    cv2.destroyAllWindows()

def visualising_animation_with_10interval(transformed_data_np, color_dict, WIDTH, HEIGHT, fps=3):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(RESULT_PATH, fourcc, fps, (WIDTH, HEIGHT))
    
    # 从transformed_data_np中获取帧数列，并按照间隔获取唯一帧数
    unique_frame_numbers = np.unique(transformed_data_np[:, 0].astype(int))

    # 存储轨迹的字典，键是 (cls_id, obj_id) 的元组
    trails_dict = {}

    # 创建动画
    for frame_number in unique_frame_numbers:  # 遍历数据集中的帧号
        frame_data = transformed_data_np[transformed_data_np[:, 0].astype(int) == frame_number]
        background = cv2.imread(BACKGROUND_PATH)
        
        # 处理当前帧的目标
        current_trails_dict = {}
        for data in frame_data:
            _, x, y, cls_id, obj_id = data
            key = (cls_id, obj_id)  # 创建键
            color = color_dict[cls_id]
            
            # 绘制目标点和文本
            position = (int(float(x)), int(float(y)))
            cv2.circle(background, position, 5, color, -1)
            cv2.putText(background, f'{cls_id} {obj_id}', (position[0] + 10, position[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            
            # 更新或添加轨迹点
            if key not in trails_dict:
                trails_dict[key] = []
            trails_dict[key].append(position)
            
            # 将当前帧的目标添加到current_trails_dict中
            current_trails_dict[key] = trails_dict[key]

        # 绘制所有轨迹
        for key, trail in trails_dict.items():
            for i in range(1, len(trail)):
                cv2.line(background, trail[i - 1], trail[i], color_dict[key[0]], 2)

        # 更新轨迹字典为当前帧的目标
        trails_dict = current_trails_dict
        
        # 显示当前帧
        cv2.imshow('Projective animation with 10 interval', background)
        
        # 写入当前帧到视频文件
        video_writer.write(background)
        
        # 按 'q' 或 'Esc' 键退出
        if cv2.waitKey(1) & 0xFF in [ord('q'), 27]:
            break
    
    # 释放资源并关闭窗口
    video_writer.release()
    cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    WIDTH, HEIGHT = get_img_size(BACKGROUND_PATH)
    # Dictionary to store the trail points of each object (choose clolors that you want:https://tool.oschina.net/commons?type=3)
    color_dict = {'person': (255, 165, 0), 'car': (238, 221, 130), 'bus': (255, 0, 0),
                  'truck': (139, 117, 0), 'bicycle': (0, 0, 128), 'motorbike': (0, 191, 255)}
    dataset = read_dataset(DATASET_PATH)
    if not dataset:
        print("No data found in the txt file or dataset is empty.")
        exit()
    coord = Coordinate(0, 0)

    numerical_data = np.asarray([row[:6] for row in dataset], dtype=np.float32)
    cls_ids = np.asarray([row[-1] for row in dataset])

    f_smoothing = []
    for data_row, cls_id in zip(numerical_data, cls_ids):
        frame_number, obj_id, x1, y1, w, h = data_row
        cx, cy, cls_id, obj_id = coord.centre_pos(x1, y1, w, h, cls_id, obj_id)
        f_smoothing.append([int(frame_number), cx, cy, cls_id, int(obj_id)])
    f_smoothing = np.asarray(f_smoothing)

    # 在这里，我们创建一个字典来存储每个ID目标的轨迹
    trajectories_dict = {}
    # 遍历转换后的数据
    for data in f_smoothing:
        frame_number, x, y, cls_id, obj_id = data
        # 使用 (cls_id, obj_id) 作为键
        key = (cls_id, obj_id)
        # 如果键不存在于字典中，则添加键并初始化为空列表
        if key not in trajectories_dict:
            trajectories_dict[key] = []
        # 将轨迹点添加到相应的列表中
        trajectories_dict[key].append((frame_number, (x, y)))
    # 创建一个空列表来存储平滑后的数据
    smoothed_data_np = []
    window_size = 5
    # 遍历字典中的每个轨迹
    for key, trajectory_points in trajectories_dict.items():
        # 提取原始的帧数列表
        frame_numbers = [point[0] for point in trajectory_points]
        # 提取坐标点（去除帧数）
        trajectory_coords = [point[1] for point in trajectory_points]
        # 如果轨迹长度小于窗口大小，则直接使用原始轨迹
        if len(trajectory_coords) < window_size:
            for point in trajectory_points:
                smoothed_data_np.append([point[0], point[1][0], point[1][1], key[0], key[1]])
            continue
        # 应用移动平均函数来平滑轨迹
        smoothed_trajectory = moving_average_trajectory(trajectory_coords, window_size)
        # 将平滑后的轨迹点与起始帧数和结束帧数结合
        smoothed_trajectory_full = [(frame_numbers[0], smoothed_trajectory[0])]
        smoothed_trajectory_full += zip(frame_numbers[1:-1], smoothed_trajectory[1:])
        smoothed_trajectory_full.append((frame_numbers[-1], smoothed_trajectory[-1]))
        # 将平滑后的轨迹添加到列表中
        cls_id, obj_id = key
        for frame_number, coords in smoothed_trajectory_full:
            smoothed_data_np.append([frame_number, coords[0], coords[1], cls_id, obj_id])
    # 将列表转换为 numpy 数组
    smoothed_data_np = np.array(smoothed_data_np)
    # 定义一个映射函数，将 cls_id 转换为排序权重
    cls_id_sorting_key = {'person': 1, 'car': 2, 'truck': 3, 'bicycle': 4}
    # 创建一个新数组，其中 cls_id 被其排序权重替换
    cls_id_as_numbers = np.array([cls_id_sorting_key[cls_id] for cls_id in smoothed_data_np[:, 3]])
    # 使用 lexsort 进行排序
    # 首先按照 obj_id 升序排序，然后是 cls_id 类型，最后是帧数
    sort_indices = np.lexsort((smoothed_data_np[:, -1].astype(int), cls_id_as_numbers, smoothed_data_np[:, 0].astype(int)))
    # 应用排序索引
    sorted_smoothed_data_np = smoothed_data_np[sort_indices]
    
    transformed_data = []
    for line in sorted_smoothed_data_np:
        frame_number, _, _, cls_id, obj_id = line
        coordinates = [line[1], line[2]]
        transformed_coordinates = coord.coordinates_inSatelliteMap(SCR, DEST, coordinates)
        transformed_data.append([int(frame_number), np.float32(transformed_coordinates[0][0][0]), np.float32(transformed_coordinates[0][0][1]), cls_id, int(obj_id)])
    transformed_data_np = np.asarray(transformed_data)

    visualising_animation(transformed_data_np, color_dict, WIDTH, HEIGHT)
    #visualising_animation_with_10interval(transformed_data_np, color_dict, WIDTH, HEIGHT)

In [ ]:
#print(trajectories_dict['person', '1'])
for line in trajectories_dict['truck', '1']:
    print(line)

In [ ]:
if __name__ == "__main__":
    WIDTH, HEIGHT = get_img_size(BACKGROUND_PATH)
    # Dictionary to store the trail points of each object (choose clolors that you want:https://tool.oschina.net/commons?type=3)
    color_dict = {'person': (255, 165, 0), 'car': (238, 221, 130), 'bus': (255, 0, 0),
                  'truck': (139, 117, 0), 'bicycle': (0, 0, 128), 'motorbike': (0, 191, 255)}
    dataset = read_dataset(DATASET_PATH)
    if not dataset:
        print("No data found in the txt file or dataset is empty.")
        exit()

    numerical_data = np.asarray([row[:6] for row in dataset], dtype=np.float32)
    cls_ids = np.asarray([row[-1] for row in dataset])
    coord = Coordinate(0, 0)
    
    transformed_data = []

    for data_row, cls_id in zip(numerical_data, cls_ids):
        frame_number, obj_id, x1, y1, w, h = data_row
        cx, cy, cls_id, obj_id = coord.centre_pos(x1, y1, w, h, cls_id, obj_id)
        coordinates = [cx, cy]
        transformed_coordinates = coord.coordinates_inSatelliteMap(SCR, DEST, coordinates)
        transformed_data.append([int(frame_number), np.float32(transformed_coordinates[0][0][0]), np.float32(transformed_coordinates[0][0][1]), cls_id, int(obj_id)])
    transformed_data_np = np.asarray(transformed_data)

    # 在这里，我们创建一个字典来存储每个ID目标的轨迹
    trajectories_dict = {}

    # 遍历转换后的数据
    for data in transformed_data_np:
        frame_number, x, y, cls_id, obj_id = data
        # 使用 (cls_id, obj_id) 作为键
        key = (cls_id, obj_id)
        
        # 如果键不存在于字典中，则添加键并初始化为空列表
        if key not in trajectories_dict:
            trajectories_dict[key] = []
        
        # 将轨迹点添加到相应的列表中
        trajectories_dict[key].append((frame_number, (x, y)))

    # 创建一个空列表来存储平滑后的数据
    smoothed_data_np = []
    window_size = 5

    # 遍历字典中的每个轨迹
    for key, trajectory_points in trajectories_dict.items():
        # 提取原始的帧数列表
        frame_numbers = [point[0] for point in trajectory_points]
        
        # 提取坐标点（去除帧数）
        trajectory_coords = [point[1] for point in trajectory_points]

        # 如果轨迹长度小于窗口大小，则直接使用原始轨迹
        if len(trajectory_coords) < window_size:
            for point in trajectory_points:
                smoothed_data_np.append([point[0], point[1][0], point[1][1], key[0], key[1]])
            continue

        # 应用移动平均函数来平滑轨迹
        smoothed_trajectory = moving_average_trajectory(trajectory_coords, window_size)

        # 将平滑后的轨迹点与起始帧数和结束帧数结合
        smoothed_trajectory_full = [(frame_numbers[0], smoothed_trajectory[0])]
        smoothed_trajectory_full += zip(frame_numbers[1:-1], smoothed_trajectory[1:])
        smoothed_trajectory_full.append((frame_numbers[-1], smoothed_trajectory[-1]))

        # 将平滑后的轨迹添加到列表中
        cls_id, obj_id = key
        for frame_number, coords in smoothed_trajectory_full:
            smoothed_data_np.append([frame_number, coords[0], coords[1], cls_id, obj_id])

    # 将列表转换为 numpy 数组
    smoothed_data_np = np.array(smoothed_data_np)

    # 定义一个映射函数，将 cls_id 转换为排序权重
    cls_id_sorting_key = {'person': 1, 'car': 2, 'truck': 3, 'bicycle': 4}

    # 创建一个新数组，其中 cls_id 被其排序权重替换
    cls_id_as_numbers = np.array([cls_id_sorting_key[cls_id] for cls_id in smoothed_data_np[:, 3]])

    # 使用 lexsort 进行排序
    # 首先按照 obj_id 升序排序，然后是 cls_id 类型，最后是帧数
    sort_indices = np.lexsort((smoothed_data_np[:, -1].astype(int), cls_id_as_numbers, smoothed_data_np[:, 0].astype(int)))

    # 应用排序索引
    sorted_smoothed_data_np = smoothed_data_np[sort_indices]

    visualising_animation(sorted_smoothed_data_np, color_dict, WIDTH, HEIGHT)
    #visualising_animation_with_10interval(transformed_data_np, color_dict, WIDTH, HEIGHT)